# In this Notebook we will create a csv for each partecipant in order to create a form in google forms for the experiment

Structure of the form:

For each game give randomly 10 player and ask to the partecipant which mark is the one predicted by the model **TURING TEST**.

For each game give randomly explanation or marks or true marks.

In [3]:
#import elements 
import numpy as np
import pandas as pd
import random
import json
import ast 

probability = 0.5

exp = pd.read_csv('dataset_for_experiment.csv')
df1718 = pd.read_csv('for_the_model.csv')
df1617 = pd.read_csv('for_the_model_2016_2017.csv')
df1819 = pd.read_csv('for_the_model_2018_2019.csv')

descriptions = pd.read_csv(r'C:\Users\matte\Data Science\Tesi\experiment\column_description.csv', encoding='latin-1')
teams = pd.read_json(r'C:\Users\matte\Data Science\Tesi\InputDataset\teams_new.json')
teams = teams.reindex(sorted(teams.columns), axis=1)
players = pd.read_json(r'C:\Users\matte\Data Science\Tesi\InputDataset\players_new.json')
players = players.reindex(sorted(players.columns), axis=1)
matches_2018_2019 = pd.read_json(r'C:\Users\matte\Data Science\Tesi\InputDataset\2018-2019\matches_2018-2019.json')
matches_2018_2019 = matches_2018_2019.reindex(sorted(matches_2018_2019.columns), axis=1)
list_games_to_watch = [2759775, 2759604, 2759546, 2759711, 2759678, 2759621, 2759574, 2759498, 2759709, 2759803]

frames = [df1617, df1718, df1819]
df = pd.concat(frames)
rolese = {'C' : 'Centrocampista', 'D':'Difensore', 'P':'Portiere', 'A': 'Attaccante'}

In [4]:
def create_formation_for_flask_app(list_of_matches_id, matches, players, teams):
    '''
    extract for each game for each team the formations.
    
    Params:
        list_of_matches_id : list of id in test set
        matches: dataset of season matches
        players: dataset of players
        teams: dataset of team for season
    '''
    matches_dictionary = {}
    for el in list_of_matches_id:
        subM = matches[matches['wyId'] == el]
        matches_dictionary[subM['label'].iloc[0]] = {}
        teams_sub = subM['teamsData'].iloc[0]
        for key in teams_sub.keys(): 
            for row in teams.values:
                if(int(row[8]) == int(key)):
                    #assign the name
                    team_name = row[6]
                    formation = []
                    titolari = teams_sub[key]['formation']['lineup']
                    for el in titolari:
                        player_name = players[players['wyId'] == el['playerId']]['shortName'].iloc[0]
                        formation.append(player_name)
                    riserve = teams_sub[key]['formation']['substitutions']
                    for el in riserve:
                        player_name = players[players['wyId'] == el['playerIn']]['shortName'].iloc[0]
                        formation.append(player_name)
                    matches_dictionary[subM['label'].iloc[0]][team_name] = formation
    return matches_dictionary

In [5]:
list_formation_visualization_purpouse = create_formation_for_flask_app(list_games_to_watch, matches_2018_2019, players, teams)

In [6]:
list_formation_visualization_purpouse

{'Lazio - Atalanta, 1 - 3': {'SS Lazio': ['F. Caicedo',
   'Bastos',
   'Lucas Leiva',
   'T. Strakosha',
   'Wallace',
   'Rômulo',
   'M. Parolo',
   'C. Immobile',
   'Luis Alberto',
   'A. Marušić',
   'F. Acerbi',
   'J. Correa',
   'M. Badelj',
   'Pedro Neto'],
  'Atalanta Bergamasca Calcio': ['J. Palomino',
   'J. Iličić',
   'R. Freuler',
   'B. Djimsiti',
   'M. de Roon',
   'H. Hateboer',
   'A. Masiello',
   'T. Castagne',
   'P. Gollini',
   'A. Gómez',
   'D. Zapata',
   'G. Mancini',
   'M. Pašalić',
   'M. Pessina']},
 'Fiorentina - Parma, 0 - 1': {'ACF Fiorentina': ['N. Milenković',
   'M. Benassi',
   'Edimilson Fernandes',
   'J. Veretout',
   'A. Lafont',
   'F. Chiesa',
   'G. Pezzella',
   'C. Biraghi',
   'Vitor Hugo',
   'G. Simeone',
   'M. Pjaca',
   'V. Laurini',
   'Gerson',
   'F. Ceccherini'],
  'Parma Calcio 1913': ['L. Štulac',
   'L. Rigoni',
   'L. Siligardi',
   'S. Iacoponi',
   'A. Bastoni',
   'L. Sepe',
   'A. Deiola',
   'J. Biabiany',
   'R. Gag

In [5]:
np.save(r'C:\Users\matte\Data Science\Tesi\experiment\formation_description.npy', list_formation_visualization_purpouse) 

In [7]:
def create_mapping_id_team_team_name(team):
    '''
    This function create a dictionary in which access key is the id of the team, the value is the team name
    Used inside the experiment to have player name (team, role)
    '''
    team_id_to_name = {}
    for t in team.values:
        team_id_to_name[t[8]] = t[6]
    return team_id_to_name    

In [8]:
teams = create_mapping_id_team_team_name(teams)

In [9]:
#map each columns to an explanation
def create_mapping_for_explanation(df, explanations, description):
    '''
    create an explanation for each column in the dataset.
    
    '''
    attackers = df[df['player_role_fantacalcio'] == 'A']
    defensers = df[df['player_role_fantacalcio'] == 'D']
    midfielders = df[df['player_role_fantacalcio'] == 'C']
    goalkeepers = df[df['player_role_fantacalcio'] == 'P']
    for_mean_extraction_a = attackers.describe().transpose()
    for_mean_extraction_c = midfielders.describe().transpose()
    for_mean_extraction_d = defensers.describe().transpose()
    for_mean_extraction_p = goalkeepers.describe().transpose()
    mapping_columns_names = {}
    for el in df.columns:
        mapping_columns_names[el] = {}
        mapping_columns_names[el]['mean_a'] = 0
        mapping_columns_names[el]['mean_c'] = 0
        mapping_columns_names[el]['mean_d'] = 0
        mapping_columns_names[el]['mean_p'] = 0
        mapping_columns_names[el]['description'] = ' '
    for index, value in for_mean_extraction_a.iterrows():
        mapping_columns_names[index]['mean_a'] = round(value[1],2)
    for index, value in for_mean_extraction_c.iterrows():
        mapping_columns_names[index]['mean_c'] = round(value[1],2)
    for index, value in for_mean_extraction_d.iterrows():
        mapping_columns_names[index]['mean_d'] = round(value[1],2)
    for index, value in for_mean_extraction_p.iterrows():
        mapping_columns_names[index]['mean_p'] = round(value[1],2)
    for el in description.values:
        for key in mapping_columns_names.keys():
            if(el[0] == key):
                mapping_columns_names[key]['description'] = el[1]
    return mapping_columns_names

In [10]:
mapping_exp = create_mapping_for_explanation(df, exp, descriptions)

In [11]:
def take_sub_df_on_match_label(list_of_labels, experiment_df):
    '''
    for each match label inside the list create a sub df where to work
    '''
    subDF = experiment_df[experiment_df['match_label'].isin(list_of_labels)]
    
    return subDF    

## Turing Test

In [12]:

def create_partecipant_players_marks_df_for_turing(partecipant, dataframe, ids, seed):
    '''
    This function return for the partecipant id, 10 players, with true and predicted values.
    These arrays will be used in Turing test.
    Due is turing test we want only those record that prediction are different from true values
    '''
    not_same_dataframe = dataframe[dataframe['model_prediction'] != dataframe['real_prediction']]
    partecipant_list = []
    player_list = []
    player_true_value = []
    player_predict_value = []
    for el in ids:
        team_sub_df = not_same_dataframe[not_same_dataframe['team_id'] == el]
        random_select_sub_df = team_sub_df.sample(frac= 0.5, random_state = seed)
        for row in random_select_sub_df.values:
            partecipant_list.append(partecipant)
            player_id = row[2] + ' ' + '(' + teams[row[1]] + ', ' + rolese[row[7]] + ')'
            player_list.append(player_id)
            player_true_value.append(row[5])
            player_predict_value.append(row[4])
    return partecipant_list, player_list, player_true_value, player_predict_value

def create_dataframe_for_pool_turing(list_partecipant_pool, df_pool, list_matches, seed, regular_pool=True):
    '''
    
    For each partecipant in the list, for each games the partecipant have to watch, compute some player
    for the turing test.
    
    '''
    list_partecipant = []
    list_player_name = []
    list_true_value = []
    list_pred_value = []
    
    pool_1_df_first_game = df_pool[df_pool['match_label'] == list_matches[0]]
    unique_tean_ids_first_game = pool_1_df_first_game['team_id'].unique()
    
    if(regular_pool):
        pool_1_df_second_game = df_pool[df_pool['match_label'] == list_matches[1]]
        unique_tean_ids_second_game = pool_1_df_second_game['team_id'].unique()

        pool_1_df_third_game = df_pool[df_pool['match_label'] == list_matches[2]]
        unique_tean_ids_third_game = pool_1_df_third_game['team_id'].unique()

        list_df = [pool_1_df_first_game, pool_1_df_second_game, pool_1_df_third_game]
        list_ids_unique = [unique_tean_ids_first_game, unique_tean_ids_second_game, unique_tean_ids_third_game]
    else:
        list_df = [pool_1_df_first_game]
        list_ids_unique = [unique_tean_ids_first_game]        
    
    for part in list_partecipant_pool:
        for i in range(0, len(list_df)):
            partname, playersname, playerstrue, playerspred = create_partecipant_players_marks_df_for_turing(part, list_df[i], list_ids_unique[i], seed)
            list_partecipant.extend(partname)
            list_player_name.extend(playersname)
            list_true_value.extend(playerstrue)
            list_pred_value.extend(playerspred)
    
    
    return list_partecipant, list_player_name, list_true_value, list_pred_value

## Treatment
3 scenarios:
1. give only the explanation
2. give only the predict value
3. give predict value plus explanation

In [13]:
def scenario_building_for_treatment(partecipant, dataframe, ids, description, seed):
    '''
    for each team inside a match take 5 player and with a sort of random provide 3 different scenarios
    '''
    partecipant_list = []
    player_list = []
    scenarios = []
    pred = []
    trues = []
    for el in ids:
        team_sub_df = dataframe[dataframe['team_id'] == el]
        random_select_sub_df = team_sub_df.sample(n = 5, random_state = seed)
        for row in random_select_sub_df.values:
            partecipant_list.append(partecipant)
            player_id = row[2] + ' ' + '(' + teams[row[1]] + ', ' + rolese[row[7]] + ')'
            player_list.append(player_id)
            #manage 3 scenarios
            ran = random.randint(0,2)
            if(ran == 0):
                #only explanation
                #ectract the motivations to give a full description string
                mot = row[6]
                mot = ast.literal_eval(mot)
                spieg = 'Informazioni Utili: '
                normal_lenght = len(spieg)
                access_mean = 'mean_' + row[7].lower()
                for el in mot.keys():
                    spieg += str(description[el]['description'])
                    spieg += ' '
                    spieg += str(round(mot[el],3))
                    spieg += ' (media nel ruolo ' 
                    spieg += str(description[el][access_mean])
                    spieg += '), '
                if(len(spieg) == normal_lenght):
                    spieg += 'Partita giocata nella media.'
                else:
                    position = (len(spieg) - 2)
                    spieg = spieg[:position]
                    spieg += '. '
                spieg += 'Qual è il voto che il giocatore meriterebbe secondo te?'
                scenarios.append(spieg)    
            if(ran == 1):
                #only the predict value
                spieg = 'Il voto assegnato al giocatore è %s. Qual è il voto che il giocatore meriterebbe secondo te? Puoi cambiare voto o confermare quello presente.' %(row[4])
                scenarios.append(spieg)
            if(ran == 2):
                #explanation and predict value
                #ectract the motivations to give a full description string
                mot = row[6]
                mot = ast.literal_eval(mot)
                spieg = 'Il voto assegnato al giocatore è %s perchè: ' %(row[4])
                normal_lenght = len(spieg)
                access_mean = 'mean_' + row[7].lower()
                for el in mot.keys():
                    spieg += str(description[el]['description'])
                    spieg += ' '
                    spieg += str(round(mot[el],3))
                    spieg += ' (media nel ruolo ' 
                    spieg += str(description[el][access_mean])
                    spieg += '), '
                if(len(spieg) == normal_lenght):
                    spieg += 'Partita giocata nella media.'
                else:
                    position = (len(spieg) -2)
                    spieg = spieg[:position]
                    spieg += '. '
                
                spieg += 'Qual è il voto che il giocatore meriterebbe secondo te? Puoi cambiare voto o confermare quello presente.'
                scenarios.append(spieg)  
            pred.append(row[4])
            trues.append(row[5])
                
    return partecipant_list, player_list, scenarios, pred, trues

def treatment_generation_dataframe(list_partecipant_pool, df_pool, list_matches, description, seed, regular_pool = True):
    '''
    For each partecipant, for each match compute the treatment scenario invoking the above function
    '''
    
    list_partecipant = []
    list_player_name = []
    scenario = []
    preds = []
    true_journalist = []
    
    pool_1_df_first_game = df_pool[df_pool['match_label'] == list_matches[0]]
    unique_tean_ids_first_game = pool_1_df_first_game['team_id'].unique()
    
    if(regular_pool):
        pool_1_df_second_game = df_pool[df_pool['match_label'] == list_matches[1]]
        unique_tean_ids_second_game = pool_1_df_second_game['team_id'].unique()

        pool_1_df_third_game = df_pool[df_pool['match_label'] == list_matches[2]]
        unique_tean_ids_third_game = pool_1_df_third_game['team_id'].unique()

        list_df = [pool_1_df_first_game, pool_1_df_second_game, pool_1_df_third_game]
        list_ids_unique = [unique_tean_ids_first_game, unique_tean_ids_second_game, unique_tean_ids_third_game]
    
    else:
        list_df = [pool_1_df_first_game]
        list_ids_unique = [unique_tean_ids_first_game]
    
    for part in list_partecipant_pool:
        for i in range(0, len(list_df)):
            partname, playersname, scenario_situation, pred, trues = scenario_building_for_treatment(part, list_df[i], list_ids_unique[i], description, seed)
            list_partecipant.extend(partname)
            list_player_name.extend(playersname)
            scenario.extend(scenario_situation)
            preds.extend(pred)
            true_journalist.extend(trues)
            
            
    
    return list_partecipant, list_player_name, scenario, preds, true_journalist

# 1 Pool (A_01, A_02, A_03)

Lazio - Atalanta, 1 - 3   Fiorentina - Parma, 0 - 1   Roma - Lazio, 3 - 1 

Turing

In [14]:
pool_1_matches = ['Lazio - Atalanta, 1 - 3', 'Fiorentina - Parma, 0 - 1', 'Roma - Lazio, 3 - 1']
pool_1_df = take_sub_df_on_match_label(pool_1_matches, exp)
list_partecipant_pool_1 = ['A_01', 'A_02', 'A_03']

pool_1_paertecipant, pool_1_player_name, pool_1_true_value, pool_1_pred_value = create_dataframe_for_pool_turing(list_partecipant_pool_1, pool_1_df, pool_1_matches, 17)
pool_1_dataframe_for_experimentation_turing = pd.DataFrame()
pool_1_dataframe_for_experimentation_turing['partecipant'] = pool_1_paertecipant
pool_1_dataframe_for_experimentation_turing['player_name'] = pool_1_player_name
listchoichesoption1 = []
listchoichesoption2 = []
for el in range(0, len(pool_1_pred_value)):
    if random.random() < probability:
        listchoichesoption1.append(pool_1_pred_value[el])
        listchoichesoption2.append(pool_1_true_value[el])
    else:
        listchoichesoption1.append(pool_1_true_value[el])
        listchoichesoption2.append(pool_1_pred_value[el])
pool_1_dataframe_for_experimentation_turing['option_1'] = listchoichesoption1
pool_1_dataframe_for_experimentation_turing['option_2'] = listchoichesoption2
pool_1_dataframe_for_experimentation_turing['model_prediction'] = pool_1_pred_value
pool_1_dataframe_for_experimentation_turing.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_1_turing_dataframe.csv', index=False)

In [15]:
pool_1_dataframe_for_experimentation_turing.head()

,partecipant,player_name,option_1,option_2,model_prediction
0,A_01,"IMMOBILE (SS Lazio, Attaccante)",5.0,5.5,5.5
1,A_01,"CAICEDO (SS Lazio, Attaccante)",6.0,6.5,6.0
2,A_01,"LUIS ALBERTO (SS Lazio, Centrocampista)",5.0,5.5,5.5
3,A_01,"WALLACE (SS Lazio, Difensore)",4.5,3.0,4.5
4,A_01,"GOMEZ A (Atalanta Bergamasca Calcio, Attaccante)",7.0,7.5,7.0


Treatment

In [16]:
pool_1_treatmant_part, pool_1_treatment_players, pool_1_treatment_scenarios, pool_1_treatment_preds, pool_1_treatment_journalist = treatment_generation_dataframe(list_partecipant_pool_1, pool_1_df, pool_1_matches, mapping_exp,17)
pool_1_dataframe_for_experimentation_treatment = pd.DataFrame()
pool_1_dataframe_for_experimentation_treatment['partecipant'] = pool_1_treatmant_part
pool_1_dataframe_for_experimentation_treatment['player_name'] = pool_1_treatment_players
pool_1_dataframe_for_experimentation_treatment['scenarios'] = pool_1_treatment_scenarios
pool_1_dataframe_for_experimentation_treatment['prediction'] = pool_1_treatment_preds
pool_1_dataframe_for_experimentation_treatment['journalist'] = pool_1_treatment_journalist

pool_1_dataframe_for_experimentation_treatment.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_1_treatment_dataframe.csv', index=False, encoding='utf-8-sig')

In [17]:
pool_1_dataframe_for_experimentation_treatment

,partecipant,player_name,scenarios,prediction,journalist
0,A_01,"PAROLO (SS Lazio, Centrocampista)",Il voto assegnato al giocatore è 6.5 perchè: N...,6.5,6.5
1,A_01,"LUIS ALBERTO (SS Lazio, Centrocampista)",Il voto assegnato al giocatore è 5.5. Qual è i...,5.5,5.0
2,A_01,"STRAKOSHA (SS Lazio, Portiere)",Il voto assegnato al giocatore è 6.5 perchè: N...,6.5,6.0
3,A_01,"BASTOS (SS Lazio, Difensore)",Il voto assegnato al giocatore è 5.0. Qual è i...,5.0,5.0
4,A_01,"IMMOBILE (SS Lazio, Attaccante)",Informazioni Utili: Numero di goal fatti 0 (me...,5.5,5.0
...,...,...,...,...,...
85,A_03,"FAZIO (AS Roma, Difensore)",Il voto assegnato al giocatore è 7.0. Qual è i...,7.0,5.5
86,A_03,"EL SHAARAWY (AS Roma, Attaccante)",Il voto assegnato al giocatore è 6.0. Qual è i...,6.0,5.0
87,A_03,"NZONZI (AS Roma, Centrocampista)",Il voto assegnato al giocatore è 6.0 perchè: P...,6.0,6.0
88,A_03,"KOLAROV (AS Roma, Difensore)",Il voto assegnato al giocatore è 7.0. Qual è i...,7.0,7.0


# 2 Pool (A_04, A_05, A_06)

Milan - Juventus, 0 - 2   Juventus - Internazionale, 1 - 0   Genoa - Cagliari, 1 - 1 

Turing

In [18]:
pool_2_matches = ['Milan - Juventus, 0 - 2', 'Juventus - Internazionale, 1 - 0', 'Genoa - Cagliari, 1 - 1']
pool_2_df = take_sub_df_on_match_label(pool_2_matches, exp)
list_partecipant_pool_2 = ['A_04', 'A_05', 'A_06']

pool_2_paertecipant, pool_2_player_name, pool_2_true_value, pool_2_pred_value = create_dataframe_for_pool_turing(list_partecipant_pool_2, pool_2_df, pool_2_matches, 17)
pool_2_dataframe_for_experimentation_turing = pd.DataFrame()
pool_2_dataframe_for_experimentation_turing['partecipant'] = pool_2_paertecipant
pool_2_dataframe_for_experimentation_turing['player_name'] = pool_2_player_name
listchoichesoption1 = []
listchoichesoption2 = []
for el in range(0, len(pool_2_pred_value)):
    if random.random() < probability:
        listchoichesoption1.append(pool_2_pred_value[el])
        listchoichesoption2.append(pool_2_true_value[el])
    else:
        listchoichesoption1.append(pool_2_true_value[el])
        listchoichesoption2.append(pool_2_pred_value[el])
pool_2_dataframe_for_experimentation_turing['option_1'] = listchoichesoption1
pool_2_dataframe_for_experimentation_turing['option_2'] = listchoichesoption2
pool_2_dataframe_for_experimentation_turing['model_prediction'] = pool_2_pred_value
pool_2_dataframe_for_experimentation_turing.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_2_turing_dataframe.csv', index=False)

In [19]:
pool_2_dataframe_for_experimentation_turing

,partecipant,player_name,option_1,option_2,model_prediction
0,A_04,"CUTRONE (AC Milan, Attaccante)",5.5,6.0,5.5
1,A_04,"CASTILLEJO (AC Milan, Attaccante)",5.0,5.5,5.5
2,A_04,"LAXALT (AC Milan, Centrocampista)",5.0,5.5,5.5
3,A_04,"ABATE (AC Milan, Difensore)",5.5,6.0,5.5
4,A_04,"MATUIDI (Juventus FC, Centrocampista)",6.0,6.5,6.0
...,...,...,...,...,...
58,A_06,"KOUAME' (Genoa CFC, Attaccante)",6.0,6.5,6.0
59,A_06,"PANDEV (Genoa CFC, Attaccante)",6.0,5.5,5.5
60,A_06,"BIRASCHI (Genoa CFC, Difensore)",6.5,6.0,6.0
61,A_06,"CRAGNO (Cagliari Calcio, Portiere)",7.0,7.5,7.5


Treatment

In [20]:
pool_2_treatmant_part, pool_2_treatment_players, pool_2_treatment_scenarios, pool_2_treatment_pred, pool_2_treatment_journalist = treatment_generation_dataframe(list_partecipant_pool_2, pool_2_df, pool_2_matches, mapping_exp,17)
pool_2_dataframe_for_experimentation_treatment = pd.DataFrame()
pool_2_dataframe_for_experimentation_treatment['partecipant'] = pool_2_treatmant_part
pool_2_dataframe_for_experimentation_treatment['player_name'] = pool_2_treatment_players
pool_2_dataframe_for_experimentation_treatment['scenarios'] = pool_2_treatment_scenarios
pool_2_dataframe_for_experimentation_treatment['prediction'] = pool_2_treatment_pred
pool_2_dataframe_for_experimentation_treatment['journalist'] = pool_2_treatment_journalist
pool_2_dataframe_for_experimentation_treatment.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_2_treatment_dataframe.csv', index=False, encoding='utf-8-sig')

# 3 Pool (A_07, A_08, A_09)

Frosinone - Atalanta, 0 - 5   Lazio - Parma, 4 - 1   Milan - Internazionale, 2 - 3

Turing

In [21]:
pool_3_matches = ['Frosinone - Atalanta, 0 - 5', 'Lazio - Parma, 4 - 1', 'Milan - Internazionale, 2 - 3']
pool_3_df = take_sub_df_on_match_label(pool_3_matches, exp)
list_partecipant_pool_3 = ['A_07', 'A_08', 'A_09']

pool_3_paertecipant, pool_3_player_name, pool_3_true_value, pool_3_pred_value = create_dataframe_for_pool_turing(list_partecipant_pool_3, pool_3_df, pool_3_matches, 17)
pool_3_dataframe_for_experimentation_turing = pd.DataFrame()
pool_3_dataframe_for_experimentation_turing['partecipant'] = pool_3_paertecipant
pool_3_dataframe_for_experimentation_turing['player_name'] = pool_3_player_name
listchoichesoption1 = []
listchoichesoption2 = []
for el in range(0, len(pool_3_pred_value)):
    if random.random() < probability:
        listchoichesoption1.append(pool_3_pred_value[el])
        listchoichesoption2.append(pool_3_true_value[el])
    else:
        listchoichesoption1.append(pool_3_true_value[el])
        listchoichesoption2.append(pool_3_pred_value[el])
pool_3_dataframe_for_experimentation_turing['option_1'] = listchoichesoption1
pool_3_dataframe_for_experimentation_turing['option_2'] = listchoichesoption2
pool_3_dataframe_for_experimentation_turing['model_prediction'] = pool_3_pred_value
pool_3_dataframe_for_experimentation_turing.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_3_turing_dataframe.csv', index=False)

In [22]:
pool_3_dataframe_for_experimentation_turing

,partecipant,player_name,option_1,option_2,model_prediction
0,A_07,"BRIGHENTI (Frosinone Calcio, Difensore)",4.0,4.5,4.5
1,A_07,"CASSATA (Frosinone Calcio, Centrocampista)",6.0,5.0,5.0
2,A_07,"MOLINARO (Frosinone Calcio, Difensore)",4.5,5.0,4.5
3,A_07,"CAMPBELL (Frosinone Calcio, Attaccante)",4.5,5.0,5.0
4,A_07,"PINAMONTI (Frosinone Calcio, Attaccante)",5.0,5.5,5.0
5,A_07,"DJIMSITI (Atalanta Bergamasca Calcio, Difensore)",7.0,6.5,7.0
6,A_07,"KULUSEVSKI (Atalanta Bergamasca Calcio, Centro...",6.5,6.0,6.5
7,A_07,"PESSINA (Atalanta Bergamasca Calcio, Centrocam...",7.0,6.5,7.0
8,A_07,"MANCINI G (Atalanta Bergamasca Calcio, Difensore)",7.5,7.0,7.0
9,A_07,"SEPE (Parma Calcio 1913, Portiere)",4.5,6.5,6.5


Treatment

In [23]:
pool_3_treatmant_part, pool_3_treatment_players, pool_3_treatment_scenarios, pool_3_tratment_pred, pool_3_treatment_journalist = treatment_generation_dataframe(list_partecipant_pool_3, pool_3_df, pool_3_matches, mapping_exp,17)
pool_3_dataframe_for_experimentation_treatment = pd.DataFrame()
pool_3_dataframe_for_experimentation_treatment['partecipant'] = pool_3_treatmant_part
pool_3_dataframe_for_experimentation_treatment['player_name'] = pool_3_treatment_players
pool_3_dataframe_for_experimentation_treatment['scenarios'] = pool_3_treatment_scenarios
pool_3_dataframe_for_experimentation_treatment['prediction'] = pool_3_tratment_pred
pool_3_dataframe_for_experimentation_treatment['journalist'] = pool_3_treatment_journalist
pool_3_dataframe_for_experimentation_treatment.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_3_treatment_dataframe.csv', index=False, encoding='utf-8-sig')

In [24]:
pool_3_dataframe_for_experimentation_treatment

,partecipant,player_name,scenarios,prediction,journalist
0,A_07,"MAIELLO (Frosinone Calcio, Centrocampista)",Il voto assegnato al giocatore è 5.0. Qual è i...,5.0,5.0
1,A_07,"BEGHETTO (Frosinone Calcio, Centrocampista)",Il voto assegnato al giocatore è 5.0 perchè: D...,5.0,6.0
2,A_07,"SPORTIELLO (Frosinone Calcio, Portiere)",Il voto assegnato al giocatore è 6.5. Qual è i...,6.5,6.5
3,A_07,"GOLDANIGA (Frosinone Calcio, Difensore)",Informazioni Utili: Differenza reti -5 (media ...,5.0,3.5
4,A_07,"MOLINARO (Frosinone Calcio, Difensore)",Informazioni Utili: Differenza reti -5 (media ...,4.5,5.0
...,...,...,...,...,...
85,A_09,"SKRINIAR (FC Internazionale Milano, Difensore)",Informazioni Utili: Partita giocata nella medi...,6.0,6.5
86,A_09,"GAGLIARDINI (FC Internazionale Milano, Centroc...",Il voto assegnato al giocatore è 6.0. Qual è i...,6.0,6.0
87,A_09,"ASAMOAH (FC Internazionale Milano, Difensore)",Informazioni Utili: Partita giocata nella medi...,6.0,6.5
88,A_09,"VECINO (FC Internazionale Milano, Centrocampista)",Informazioni Utili: Numero di goal fatti 1 (me...,7.0,7.0


# 4 Pool (A_10, A_11, A_12)

Frosinone - Roma, 2 - 3

Turing

In [25]:
pool_4_matches = ['Frosinone - Roma, 2 - 3']
pool_4_df = take_sub_df_on_match_label(pool_4_matches, exp)
list_partecipant_pool_4 = ['A_10', 'A_11', 'A_12']

pool_4_paertecipant, pool_4_player_name, pool_4_true_value, pool_4_pred_value = create_dataframe_for_pool_turing(list_partecipant_pool_4, pool_4_df, pool_4_matches, 17, False)
pool_4_dataframe_for_experimentation_turing = pd.DataFrame()
pool_4_dataframe_for_experimentation_turing['partecipant'] = pool_4_paertecipant
pool_4_dataframe_for_experimentation_turing['player_name'] = pool_4_player_name
listchoichesoption1 = []
listchoichesoption2 = []
for el in range(0, len(pool_4_pred_value)):
    if random.random() < probability:
        listchoichesoption1.append(pool_4_pred_value[el])
        listchoichesoption2.append(pool_4_true_value[el])
    else:
        listchoichesoption1.append(pool_4_true_value[el])
        listchoichesoption2.append(pool_4_pred_value[el])
pool_4_dataframe_for_experimentation_turing['option_1'] = listchoichesoption1
pool_4_dataframe_for_experimentation_turing['option_2'] = listchoichesoption2
pool_4_dataframe_for_experimentation_turing['model_prediction'] = pool_4_pred_value
pool_4_dataframe_for_experimentation_turing.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_4_turing_dataframe.csv', index=False)

In [26]:
pool_4_dataframe_for_experimentation_turing

,partecipant,player_name,option_1,option_2,model_prediction
0,A_10,"SALAMON (Frosinone Calcio, Difensore)",5.5,6.0,5.5
1,A_10,"GOLDANIGA (Frosinone Calcio, Difensore)",5.0,4.0,5.0
2,A_10,"CIANO (Frosinone Calcio, Attaccante)",7.5,7.0,7.0
3,A_10,"CAPUANO (Frosinone Calcio, Difensore)",5.0,5.5,5.5
4,A_10,"KOLAROV (AS Roma, Difensore)",6.0,5.5,6.0
5,A_10,"MANOLAS (AS Roma, Difensore)",6.5,6.0,6.0
6,A_10,"DZEKO (AS Roma, Attaccante)",7.5,8.0,7.5
7,A_10,"SANTON (AS Roma, Difensore)",5.5,6.0,6.0
8,A_11,"SALAMON (Frosinone Calcio, Difensore)",6.0,5.5,5.5
9,A_11,"GOLDANIGA (Frosinone Calcio, Difensore)",5.0,4.0,5.0


Treatment

In [27]:
pool_4_treatmant_part, pool_4_treatment_players, pool_4_treatment_scenarios, pool_4_treatment_pred, pool_4_treatment_journalist = treatment_generation_dataframe(list_partecipant_pool_4, pool_4_df, pool_4_matches, mapping_exp,17, False)
pool_4_dataframe_for_experimentation_treatment = pd.DataFrame()
pool_4_dataframe_for_experimentation_treatment['partecipant'] = pool_4_treatmant_part
pool_4_dataframe_for_experimentation_treatment['player_name'] = pool_4_treatment_players
pool_4_dataframe_for_experimentation_treatment['scenarios'] = pool_4_treatment_scenarios
pool_4_dataframe_for_experimentation_treatment['prediction'] = pool_4_treatment_pred
pool_4_dataframe_for_experimentation_treatment['journalist'] = pool_4_treatment_journalist
pool_4_dataframe_for_experimentation_treatment.to_csv(r'C:\Users\matte\Data Science\Tesi\experiment\prova_pool_4_treatment_dataframe.csv', index=False, encoding='utf-8-sig')

In [28]:
pool_4_dataframe_for_experimentation_treatment

,partecipant,player_name,scenarios,prediction,journalist
0,A_10,"CASSATA (Frosinone Calcio, Centrocampista)",Il voto assegnato al giocatore è 5.5 perchè: E...,5.5,6.0
1,A_10,"CIOFANI D (Frosinone Calcio, Attaccante)",Informazioni Utili: Numero di goal fatti 0 (me...,5.5,5.5
2,A_10,"SPORTIELLO (Frosinone Calcio, Portiere)",Il voto assegnato al giocatore è 6.5. Qual è i...,6.5,5.0
3,A_10,"SALAMON (Frosinone Calcio, Difensore)",Il voto assegnato al giocatore è 5.5. Qual è i...,5.5,6.0
4,A_10,"GOLDANIGA (Frosinone Calcio, Difensore)",Informazioni Utili: Numero dei goal sofferti d...,5.0,4.0
5,A_10,"DE ROSSI (AS Roma, Centrocampista)",Informazioni Utili: Passaggi effettuati rispet...,6.0,6.0
6,A_10,"PELLEGRINI LO (AS Roma, Centrocampista)",Il voto assegnato al giocatore è 7.0. Qual è i...,7.0,7.0
7,A_10,"OLSEN (AS Roma, Portiere)",Informazioni Utili: Numero dei goal sofferti d...,6.5,5.5
8,A_10,"KOLAROV (AS Roma, Difensore)",Il voto assegnato al giocatore è 6.0 perchè: D...,6.0,5.5
9,A_10,"MANOLAS (AS Roma, Difensore)",Il voto assegnato al giocatore è 6.0. Qual è i...,6.0,6.5
